**Importing needed libraries and loading file**

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
file= pd.read_csv('C:\\Users\\Admin\\Downloads\\TMDB_movie_dataset_v11.csv')


**Filtering movies released on or after January 1, 2010, and produced in the USA and/or UK**
*including co-productions with other countries.*

In [6]:
df_filtered= file[file['release_date']>='2010-01-01']

In [7]:
df_filtered =df_filtered[df_filtered['production_countries'].str.contains("United States of America|United Kingdom", na=False)]

**Making a copy of the imported file,** *copies of this file will be made at different points in this code*

In [8]:
movies_df=df_filtered.copy()

**Eye-balling the data**

In [9]:
#setting display options to be able to view all columns 
pd.set_option('display.max_columns', None)

In [10]:
movies_df.head(7)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,https://www.warnerbros.com/movies/inception,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,165000000,http://www.interstellarmovie.net/,tt0816692,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,220000000,https://www.marvel.com/movies/the-avengers,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
5,293660,Deadpool,7.606,28894,Released,2016-02-09,783100000,108,False,/en971MEXui9diirXlogOrPKmsEn.jpg,58000000,https://www.20thcenturystudios.com/movies/dead...,tt1431045,en,Deadpool,The origin story of former Special Forces oper...,72.735,/zq8Cl3PNIDGU3iWNRoc5nEZ6pCe.jpg,Witness the beginning of a happy ending.,"Action, Adventure, Comedy","20th Century Fox, The Donners' Company, Genre ...",United States of America,English,"superhero, anti hero, mercenary, based on comi..."
6,299536,Avengers: Infinity War,8.255,27713,Released,2018-04-25,2052415039,149,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,300000000,https://www.marvel.com/movies/avengers-infinit...,tt4154756,en,Avengers: Infinity War,As the Avengers and their allies have continue...,154.340,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,An entire universe. Once and for all.,"Adventure, Action, Science Fiction",Marvel Studios,United States of America,"English, Xhosa","sacrifice, magic, superhero, based on comic, s..."
8,118340,Guardians of the Galaxy,7.906,26638,Released,2014-07-30,772776600,121,False,/uLtVbjvS1O7gXL8lUOwsFOH4man.jpg,170000000,http://marvel.com/guardians,tt2015381,en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",33.255,/r7vmZjiyZw9rpJMQJdXpjgiCOk9.jpg,All heroes start somewhere.,"Action, Science Fiction, Adventure",Marvel Studios,United States of America,English,"spacecraft, based on comic, space, orphan, adv..."
13,68718,Django Unchained,8.171,24672,Released,2012-12-25,425368238,165,False,/5Lbm0gpFDRAPIV1Cth6ln9iL1ou.jpg,100000000,http://www.unchainedmovie.com,tt1853728,en,Django Unchained,"With the help of a German bounty hunter, a fre...",54.224,/7oWY8VDWW7thTzWh3OKYRkWUlD5.jpg,"Life, liberty and the pursuit of vengeance.","Drama, Western","The Weinstein Company, Columbia Pictures",United States of America,"English, French, German","rescue, friendship, bounty hunter, texas, slav..."


In [11]:
#checking datatypes of each column to ensure consistency of column dtypes with data they contain
print(movies_df.dtypes)

id                        int64
title                    object
vote_average            float64
vote_count                int64
status                   object
release_date             object
revenue                   int64
runtime                   int64
adult                      bool
backdrop_path            object
budget                    int64
homepage                 object
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity              float64
poster_path              object
tagline                  object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
dtype: object


In [12]:
#from the dtypes only the release_date is inconsistent with the content of the column
#converting release_date to DateTime
movies_df['release_date']= pd.to_datetime(movies_df['release_date'])

In [13]:
print(movies_df['release_date'].dtypes)

datetime64[ns]


**Dropping columns not needed for this analysis and not needed when handling missing data**

In [14]:
movies_df= movies_df.drop(['backdrop_path','imdb_id','poster_path','keywords','tagline','id'], axis=1)

*Columns left are either needed for the analysis or are seen to contain information that might be needed when handling missing data, many of these columns however, will be dropped subsequently after handling missing data.*

In [15]:
#dropping duplicate rows 
movies_df= movies_df.drop_duplicates()

In [16]:
movies_df2=movies_df.copy()

**Resolving missing data, inconsistent data and dropping unwanted rows**

In [17]:
#summing missing data per column
print(movies_df.isnull().sum())

title                       0
vote_average                0
vote_count                  0
status                      0
release_date                0
revenue                     0
runtime                     0
adult                       0
budget                      0
homepage                72576
original_language           0
original_title              0
overview                 2424
popularity                  0
genres                  31129
production_companies    24040
production_countries        0
spoken_languages        20989
dtype: int64


In [18]:
#checking number of 0s contained in columns with integer or float dtype
print(movies_df[['budget', 'vote_average', 'vote_count', 'revenue', 'runtime']].eq(0).sum())


budget          89369
vote_average    54364
vote_count      54325
revenue         96471
runtime          6479
dtype: int64


*Vote_average, budget, revenue on closer look contain lots of 0s which are considered missing values in this analysis,
given the importance of these columns in the dataset and large number of rows with 0s, 
substituting the missing numbers with statistical values will lead to mispresenation of data and error,
therefore, rows with 0 revenue, 0 vote_average and 0 budget are dropped to reduce data volume and ensure data integrity.
these are not treated as missing data, rather as information that will hamper data integrity.*

In [19]:
movies_df2=movies_df.copy()

In [20]:
movies_df = movies_df.drop(movies_df[(movies_df['revenue'] == 0) | (movies_df['budget'] == 0)|(movies_df['vote_average'] == 0)].index)

*Homepage, overview,original_title and spoken_languages are only considered valuable for handling missing data in this dataset, 
therefore missing data contained within them will not be resolved.*


*Some rows are seen to contain trailers, wrestling clips, adult content and youtube skits with common identifiable tags and runtime lower than 15mins, these are considered unwanted for these analysis,
these are dropped filtering by the tags and by runtime*

In [21]:
# Dropping rows containing wrestling videos (case insensitive) using their various unique production_company tags
movies_df = movies_df[~movies_df['production_companies'].str.contains('queens of combat|wsu|wwe|wrestle|wrestling|ctRl|fight', case=False, na=False)]

In [22]:
#dropping rows with runtime lower than 15mins
movies_df = movies_df[movies_df['runtime'] >= 15]

In [23]:
print(len(movies_df))

2384


*Vote_average is seen to be the movie viewer rating calculated relative to the vote_count, vote_count being the number of votes made per movie
I dropped vote counts less than 3 to ensure a more reliable and representative rating system. 
With very few votes, ratings can be easily skewed by individual biases or outliers, leading to inaccurate representations of a movie's quality. 
By setting a minimum threshold of 5 votes, I increase confidence in the ratings, enable fairer comparisons, and include movies with low but meaningful engagement*

In [24]:
movies_df = movies_df[movies_df['vote_count'] >= 5]

In [25]:
print(len(movies_df))

2283


In [26]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [27]:
#Confirming columns still with missing data
print(movies_df.isnull().sum())

title                     0
vote_average              0
vote_count                0
status                    0
release_date              0
revenue                   0
runtime                   0
adult                     0
budget                    0
homepage                685
original_language         0
original_title            0
overview                  1
popularity                0
genres                    0
production_companies     10
production_countries      0
spoken_languages          3
dtype: int64


In [28]:
#printing titles of movies with missing 'production_companies'
print(movies_df[movies_df['production_companies'].isnull()][['title']])

                         title
18458          Lost in the Sun
21957          The Killing Jar
25955               Repentance
29031    2016: Obama's America
29618            Higher Ground
30614  Saint John of Las Vegas
32406           Crooked Arrows
36530       Meet Monica Velour
39713            The Afflicted
67596         American Dresser


*Missing 'Production_companies' are manually replaced with corresponding data obtained, verified and confirmed from trusted sources.
Where correct and verfied corresponding data are not found, 'Unknown' is used*

In [29]:
# creating a dictionary from the list of companies manually gathered
production_companies_dict = {
    "Lost in the Sun": "Floren Shieh Productions",
    "The Killing Jar": "Morningstar Pictures",
    "Repentance": "Codeblack Films, JuntoBox Films",
    "Saint John of Las Vegas": "IndieVest Pictures, Circle of Confusion",
    "Crooked Arrows": "Sports Studio"}

In [30]:
# Replacing NaN rows in production_companies column with known production companies
movies_df.loc[movies_df['title'].isin(production_companies_dict.keys()), 'production_companies'] = movies_df['title'].map(production_companies_dict)

In [32]:
#Replacing unverified Production company missing rows with 'Unknown'
movies_df['production_companies'] = movies_df['production_companies'].fillna('Unknown')

In [33]:
print(movies_df.isnull().sum())

title                     0
vote_average              0
vote_count                0
status                    0
release_date              0
revenue                   0
runtime                   0
adult                     0
budget                    0
homepage                685
original_language         0
original_title            0
overview                  1
popularity                0
genres                    0
production_companies      0
production_countries      0
spoken_languages          3
dtype: int64


In [33]:
print(movies_df['original_language'].unique())

['en' 'ko' 'es' 'id' 'fr' 'ja' 'it' 'ar' 'pl' 'fi' 'hi' 'da' 'ru' 'de'
 'th' 'he' 'ml' 'ur']


*writing 'original_language' in full with data manually obtained from the 'spoken_languages' column*

In [34]:
language_map = {
    'ko': 'Korean',
    'es': 'Spanish',
    'id': 'Indonesian',
    'fr': 'French',
    'ja': 'Japanese',
    'it': 'Italian',
    'fi': 'Finnish',
    'ar': 'Arabic',
    'pl': 'Polish',
    'hi': 'Hindi',
    'da': 'Danish',
    'ru': 'Russian',
    'de': 'German',
    'th': 'Thai',
    'he': 'Hebrew',
    'ml': 'Malayalam',
    'ur': 'Urdu',
    'en': 'English'
}

movies_df['original_language'] = movies_df['original_language'].map(language_map)

In [35]:
print(movies_df['original_language'].unique())

['English' 'Korean' 'Spanish' 'Indonesian' 'French' 'Japanese' 'Italian'
 'Arabic' 'Polish' 'Finnish' 'Hindi' 'Danish' 'Russian' 'German' 'Thai'
 'Hebrew' 'Malayalam' 'Urdu']


In [36]:
#filling missing languages in spoken_languages column
movies_df['spoken_languages'] = movies_df['spoken_languages'].fillna(movies_df['original_language'])

In [37]:
print(len(movies_df))

2283


In [38]:
movies_df2= movies_df.copy()

**Creating release_year and release_month columns off the release_date column for ease in analysis**

In [39]:
movies_df['release_month'] = movies_df['release_date'].dt.month
movies_df['release_year'] = movies_df['release_date'].dt.year
print(movies_df[['release_year','release_month']].head(5))

   release_year  release_month
0          2010              7
1          2014             11
4          2012              4
5          2016              2
6          2018              4


In [40]:
#formatting the month as a fully writen month name
movies_df['release_month'] = movies_df['release_date'].dt.strftime('%B')
print(movies_df[['release_month']].head(5))

  release_month
0          July
1      November
4         April
5      February
6         April


In [41]:
movies_df2=movies_df.copy()

**Dropping more unneeded columns previously used in filling missing data**

In [42]:
movies_df = movies_df.drop(['vote_count', 'status', 'homepage', 'original_title', 'overview','release_date','adult'], axis=1)


**Renaming columns**

In [53]:
movies_df = movies_df.rename(columns={
    'original_language': 'Language',
    'spoken_languages': 'Spoken Languages',
    'release_year': 'Release Year',
    'production_companies': 'Production Companies',
    'budget': 'Production Budget(USD)',
    'revenue': 'Revenue(USD)',
    'release_month':'Release Month',
    'title':'Title',
    'vote_average':'Average Rating',
    'runtime':'Run Time(Mins)',
    'popularity':'Popularity',
    'genres':'Genres',
    'production_countries':'Production Countries'
    
})

**Final reformatting of table**

In [54]:
#reordering the columns
movies_df = movies_df[['Title',
                       'Release Year',
                       'Release Month',
                       'Run Time(Mins)',
                       'Production Budget(USD)',
                       'Revenue(USD)',
                       'Average Rating',
                       'Popularity',
                       'Genres',
                       'Language',
                       'Spoken Languages',
                       'Production Companies',
                       'Production Countries']]
movies_df.head(0)

,Title,Release Year,Release Month,Run Time(Mins),Production Budget(USD),Revenue(USD),Average Rating,Popularity,Genres,Language,Spoken Languages,Production Companies,Production Countries


**Resetting Index**

In [49]:
movies_df = movies_df.reset_index(drop=True)

In [51]:
#setting index to start from 1
movies_df = movies_df.set_index(movies_df.index + 1)

In [52]:
#Rounding floats to 2 decimal places
movies_df['Average Rating'] = movies_df['Average Rating'].round(2)
movies_df['Popularity'] = movies_df['Popularity'].round(2)
movies_df.head(10)

,Title,Release Year,Release Month,Run Time(Mins),Production Budget(USD),Revenue(USD),Average Rating,Popularity,Genres,Language,Spoken Languages,Production Companies,Production Countries
1,Inception,2010,July,148,160000000,825532764,8.36,83.95,"Action, Science Fiction, Adventure",English,"English, French, Japanese, Swahili","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America"
2,Interstellar,2014,November,169,165000000,701729206,8.42,140.24,"Adventure, Drama, Science Fiction",English,English,"Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America"
3,The Avengers,2012,April,143,220000000,1518815515,7.71,98.08,"Science Fiction, Action, Adventure",English,"English, Hindi, Russian",Marvel Studios,United States of America
4,Deadpool,2016,February,108,58000000,783100000,7.61,72.74,"Action, Adventure, Comedy",English,English,"20th Century Fox, The Donners' Company, Genre ...",United States of America
5,Avengers: Infinity War,2018,April,149,300000000,2052415039,8.26,154.34,"Adventure, Action, Science Fiction",English,"English, Xhosa",Marvel Studios,United States of America
6,Guardians of the Galaxy,2014,July,121,170000000,772776600,7.91,33.26,"Action, Science Fiction, Adventure",English,English,Marvel Studios,United States of America
7,Django Unchained,2012,December,165,100000000,425368238,8.17,54.22,"Drama, Western",English,"English, French, German","The Weinstein Company, Columbia Pictures",United States of America
8,Avengers: Endgame,2019,April,181,356000000,2800000000,8.26,91.76,"Adventure, Science Fiction, Action",English,"English, Japanese, Xhosa",Marvel Studios,United States of America
9,Joker,2019,October,122,55000000,1074458282,8.17,54.52,"Crime, Thriller, Drama",English,English,"Warner Bros. Pictures, Joint Effort, Village R...","Canada, United States of America"
10,Shutter Island,2010,February,138,80000000,294800000,8.20,56.60,"Drama, Thriller, Mystery",English,"English, German","Phoenix Pictures, Paramount, Appian Way, Sikel...",United States of America


In [47]:
movies_df.to_csv('C:\\Users\\Admin\\Downloads\\Movies-Cleaned-df.csv', index= False)